# RAG 파이프라인에서 VLM의 역할

RAG(Retrieval-Augmented Generation) 파이프라인에서 VLM(Vision-Language Model)은 텍스트 중심의 한계를 넘어 멀티모달리티를 도입하여 정확하고 풍부한 답변을 생성하는 데 핵심적인 역할을 수행합니다. 기존의 RAG는 주로 텍스트 데이터만을 다루었지만, VLM을 활용하면 이미지, 차트, 표, 그래프 등 시각적 정보를 이해하고 이를 답변 생성에 활용할 수 있게 됩니다.

## 1. VLM의 주요 역할과 사용 사례

### 문서 파싱 및 정보 추출
복잡한 문서(예: 논문, 보고서, PPT 등)에서 텍스트뿐만 아니라 표, 이미지, 차트 등 시각적 요소를 분석하여 텍스트 정보로 변환합니다. 예를 들어, VLM은 PDF 내의 차트를 읽고 해당 차트의 내용을 설명하는 캡션을 생성하여 RAG 파이프라인이 이를 검색 가능한 텍스트로 활용하게 합니다.

### 멀티모달 임베딩 생성
텍스트와 이미지를 하나의 의미 있는 벡터 공간에 임베딩하여, 텍스트 쿼리에 대한 답변을 검색할 때 관련 이미지도 함께 찾아낼 수 있도록 합니다. 이를 통해 텍스트만으로는 전달하기 어려운 시각적 정보가 풍부한 답변을 제공할 수 있습니다.

### 검색 및 답변 생성
사용자의 질의가 이미지와 관련된 경우, VLM은 검색된 여러 이미지 또는 문서 페이지를 바탕으로 답변을 생성합니다. 예를 들어, 여러 페이지의 이미지들을 하나로 합치거나(페이지 연결), 각 페이지별로 VLM이 답변을 생성한 후 가장 신뢰도가 높은 답변을 선택하는 방식(가중치 기반 선택)으로 답변의 정확도를 높입니다.

### 정확성 및 신뢰성 향상
시각적 정보를 직접 분석하고 활용함으로써, 텍스트만으로는 알 수 없는 맥락이나 세부 정보를 포착하여 답변의 환각(hallucination) 현상을 줄이고 신뢰성을 향상시킵니다.

## 2. VLM 기반 RAG의 특징 (VisRAG)

- **텍스트-이미지 통합**: VLM 기반의 RAG는 **VisRAG(Vision-Based Retrieval-Augmented Generation)**로 불리기도 하며, 텍스트와 이미지를 모두 아우르는 멀티모달 검색을 가능하게 합니다.
- **학습 효율성 및 일반화 능력**: 기존 텍스트 기반 RAG보다 더 나은 학습 데이터 효율성과 일반화 능력을 보여주며, 다양한 유형의 문서에 대해 견고한(robust) 성능을 입증하고 있습니다.
- **새로운 표준**: 향후 기존의 텍스트 기반 RAG를 대체할 차세대 표준으로 자리매김할 가능성이 높습니다.

## 3. Docling VLM Pipeline 예제

아래 예제는 Docling의 VLM Pipeline을 사용하여 PDF 문서를 처리하는 방법을 보여줍니다. 이 파이프라인은 PDF의 시각적 요소를 이해하고 마크다운으로 변환하는 과정을 수행합니다.

### 필요한 라이브러리 임포트

Docling의 VLM 파이프라인을 사용하기 위해 필요한 모듈들을 임포트합니다:
- `VlmPipeline`: 시각적 언어 모델을 사용한 문서 처리 파이프라인
- `DocumentConverter`: 다양한 형식의 문서를 변환하는 메인 클래스
- `PdfFormatOption`: PDF 처리를 위한 옵션 설정

In [1]:
from docling.datamodel import vlm_model_specs
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    VlmPipelineOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.pipeline.vlm_pipeline import VlmPipeline

### PDF 소스 설정

처리할 PDF 문서의 URL을 설정합니다. 이 예제에서는 arXiv의 논문 PDF를 사용합니다.

In [2]:
# 처리할 PDF 문서의 URL 설정
# 이 예제에서는 arXiv의 논문 PDF를 사용합니다
source = "https://arxiv.org/pdf/2501.17887"

### DocumentConverter 설정

PDF 문서를 처리하기 위한 DocumentConverter를 설정합니다. 여기서 중요한 점은 `VlmPipeline`을 사용하도록 지정하는 것입니다. 이를 통해 PDF의 시각적 요소(이미지, 차트, 표 등)를 이해하고 텍스트로 변환할 수 있습니다.

In [3]:
# VLM 파이프라인을 사용하는 DocumentConverter 생성
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_cls=VlmPipeline,  # VLM 파이프라인 사용 지정
        ),
    }
)

### PDF 문서 변환 실행

이제 실제로 PDF 문서를 변환합니다. 이 과정에서 VLM이 다음과 같은 작업을 수행합니다:

1. **PDF 다운로드 및 페이지 분석**: 지정된 URL에서 PDF를 다운로드하고 각 페이지를 이미지로 변환
2. **시각적 요소 인식**: 텍스트, 이미지, 표, 차트, 수식 등을 식별
3. **구조 분석**: 제목, 단락, 목록 등의 문서 구조를 파악
4. **텍스트 추출 및 변환**: 시각적 정보를 텍스트로 변환하고 마크다운 형식으로 구조화

**참고**: 첫 실행 시 SmolDocling 모델을 다운로드하므로 시간이 걸릴 수 있습니다.

In [ ]:
# PDF 문서를 VLM 파이프라인으로 변환
# 이 과정에서 SmolDocling 모델이 PDF의 시각적 요소를 분석합니다
doc = converter.convert(source=source).document

### 마크다운 형식으로 결과 출력

변환된 문서를 마크다운 형식으로 출력합니다. VLM 파이프라인을 통해 처리된 결과에는 다음이 포함됩니다:

- **구조화된 텍스트**: 제목, 단락, 목록 등이 마크다운 형식으로 변환
- **표 데이터**: PDF의 표가 마크다운 테이블로 변환
- **이미지 설명**: 차트나 그래프의 내용이 텍스트로 설명
- **수식**: 수학 공식이 LaTeX 형식으로 변환

이렇게 변환된 마크다운은 RAG 시스템에서 검색 가능한 텍스트로 활용할 수 있습니다.

In [ ]:
# 변환된 문서를 마크다운 파일로 저장
# VLM이 분석한 모든 시각적 요소가 텍스트로 변환되어 파일에 저장됩니다
markdown_content = doc.export_to_markdown()
with open('converted_document.md', 'w', encoding='utf-8') as f:
    f.write(markdown_content)
print("마크다운 파일이 'converted_document.md'로 저장되었습니다.")

## 결론

이 예제를 통해 VLM 파이프라인이 어떻게 PDF 문서의 시각적 정보를 이해하고 구조화된 텍스트로 변환하는지 확인할 수 있습니다. 이러한 기능은 RAG 시스템에서 다음과 같은 이점을 제공합니다:

1. **완전한 정보 추출**: 텍스트뿐만 아니라 이미지, 차트, 표의 정보까지 활용
2. **구조 보존**: 문서의 계층 구조와 레이아웃 정보 유지
3. **검색 효율성**: 모든 정보가 텍스트로 변환되어 검색 가능
4. **정확성 향상**: 시각적 맥락을 이해하여 더 정확한 답변 생성

VLM 기반 RAG는 기존 텍스트 기반 시스템의 한계를 극복하고, 더욱 풍부하고 정확한 정보 검색 및 생성을 가능하게 합니다.